# NLP Assignment 2 
- POS tagging using Viterbi Algorithm
- Includes 2 implementations

Conclusion
- Standard viterbi algorithm accuracy ~ 0.8
- Modified viterbi algorithm accuracy ~ 0.9

## Harshal Deshpande - BT18CSE079

In [1]:
# Importing necessary libraries

import nltk
import numpy as np 
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
# Reading the Treebank tagged sentences corpus

nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
nltk_data = nltk_data[:1000] # Using first 1000 sentences for reducing computation time

In [3]:
# Print data for analysis
nltk_data[:5]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

In [4]:
# Splitting into train and test sets

# seed to get same split on every execution
random.seed(1234)
train_set, test_set = train_test_split(nltk_data, test_size=0.05)

print("Training Set Length -", len(train_set))
print("Test Set Length -", len(test_set))
print("-" * 120)
print("Training Data Glimpse -\n")
print(train_set[:5])

Training Set Length - 950
Test Set Length - 50
------------------------------------------------------------------------------------------------------------------------
Training Data Glimpse -

[[('Reserves', 'NOUN'), ('for', 'ADP'), ('the', 'DET'), ('five', 'NUM'), ('new', 'ADJ'), ('fields', 'NOUN'), ('total', 'VERB'), ('50', 'NUM'), ('million', 'NUM'), ('barrels', 'NOUN'), ('.', '.')], [('If', 'ADP'), ('``', '.'), ('A', 'DET'), ('Wild', 'NOUN'), ('Sheep', 'NOUN'), ('Chase', 'NOUN'), ("''", '.'), ('carries', 'VERB'), ('an', 'DET'), ('implicit', 'ADJ'), ('message', 'NOUN'), ('for', 'ADP'), ('international', 'ADJ'), ('relations', 'NOUN'), (',', '.'), ('it', 'PRON'), ("'s", 'VERB'), ('that', 'ADP'), ('the', 'DET'), ('Japanese', 'NOUN'), ('are', 'VERB'), ('more', 'ADV'), ('like', 'ADP'), ('us', 'PRON'), ('than', 'ADP'), ('most', 'ADJ'), ('of', 'ADP'), ('us', 'PRON'), ('think', 'VERB'), ('0', 'X'), ('*?*', 'X'), ('.', '.')], [('Commonwealth', 'NOUN'), ('Edison', 'NOUN'), ('Co.', 'NOUN'), ('

In [5]:
# Comprehend list to get all pairs of words and tags in single list

train_pairs = [tup for sent in train_set for tup in sent]
train_pairs[:10]

[('Reserves', 'NOUN'),
 ('for', 'ADP'),
 ('the', 'DET'),
 ('five', 'NUM'),
 ('new', 'ADJ'),
 ('fields', 'NOUN'),
 ('total', 'VERB'),
 ('50', 'NUM'),
 ('million', 'NUM'),
 ('barrels', 'NOUN')]

In [6]:
# Extract all words from the training pairs

train_data_tokens = [pair[0] for pair in train_pairs]
train_data_tokens[:10]

['Reserves',
 'for',
 'the',
 'five',
 'new',
 'fields',
 'total',
 '50',
 'million',
 'barrels']

In [7]:
# Extract all tags from the training pairs

train_data_pos_tags = [pair[1] for pair in train_pairs]
train_data_pos_tags[:10]

['NOUN', 'ADP', 'DET', 'NUM', 'ADJ', 'NOUN', 'VERB', 'NUM', 'NUM', 'NOUN']

In [8]:
# building the train vocabulary to a set

training_vocabulary_set = set(train_data_tokens)

In [9]:
# building the POS tags to a set

training_pos_tag_set = set(train_data_pos_tags)

In [10]:
print("\nAvailable Tags :\n")
print(training_pos_tag_set)


Available Tags :

{'X', 'ADP', 'ADV', 'VERB', 'PRT', 'PRON', 'ADJ', 'DET', 'NOUN', 'NUM', '.', 'CONJ'}


## Compute emmision and transition probabilities

In [11]:
# Computing P(w/t) and storing in [Tags x Vocabulary] matrix. 
# This is a matrix with dimension of len(training_pos_tag_set) X len(training_vocabulary_set)

len_pos_tags = len(training_pos_tag_set)
len_vocab = len(training_vocabulary_set)

# Initialize matrix with all zeros
word_given_tag = np.zeros((len_pos_tags, len_vocab))

In [12]:
# Function to compute emission probabilitiy given word and tag

def calc_emmision_probability(word, tag, train_data = train_pairs):
    tag_list = [pair for pair in train_data if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)
    # Returns P(w/t) and P(t)

In [13]:
# Function to compute transition probabilitiy given two tags

def calc_transition_probability(t2, t1, train_data = train_pairs):
    tags = [pair[1] for pair in train_data]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)
    # Returns P(t2/t1) and P(t1)

In [14]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

transition_probabilities = np.zeros((len_pos_tags, len_pos_tags), dtype='float32')
for i, t1 in enumerate(list(training_pos_tag_set)):
    for j, t2 in enumerate(list(training_pos_tag_set)): 
        transition_probabilities[i, j] = calc_transition_probability(t2, t1)[0]/calc_transition_probability(t2, t1)[1]

transition_probabilities

array([[7.59819672e-02, 1.60978749e-01, 2.25370247e-02, 2.05408886e-01,
        1.80940121e-01, 6.18158393e-02, 1.73857044e-02, 5.15131988e-02,
        6.43915012e-02, 6.43915031e-04, 1.49388283e-01, 9.01481044e-03],
       [2.95833331e-02, 1.70833338e-02, 1.41666671e-02, 5.83333336e-03,
        1.24999997e-03, 6.12500012e-02, 1.03333332e-01, 3.27083319e-01,
        3.42500001e-01, 6.58333302e-02, 3.20833325e-02, 0.00000000e+00],
       [3.34672034e-02, 1.35207504e-01, 8.56760368e-02, 3.22623819e-01,
        1.60642564e-02, 1.74029451e-02, 1.19143240e-01, 7.36278445e-02,
        4.55153957e-02, 2.54350733e-02, 1.17804550e-01, 8.03212821e-03],
       [2.33073324e-01, 8.86115432e-02, 7.73790926e-02, 1.65678620e-01,
        3.43213715e-02, 3.96255851e-02, 6.95787817e-02, 1.31669268e-01,
        1.09828390e-01, 2.02808119e-02, 2.62090489e-02, 3.74414981e-03],
       [1.26903551e-02, 1.77664980e-02, 1.01522841e-02, 3.84517759e-01,
        1.26903551e-03, 2.41116751e-02, 7.74111673e-02, 1.05

In [15]:
# convert the matrix to data frame
transition_df = pd.DataFrame(transition_probabilities, columns = list(training_pos_tag_set), index=list(training_pos_tag_set))
transition_df

,X,ADP,ADV,VERB,PRT,PRON,ADJ,DET,NOUN,NUM,.,CONJ
X,0.075982,0.160979,0.022537,0.205409,0.180940,0.061816,0.017386,0.051513,0.064392,0.000644,0.149388,0.009015
ADP,0.029583,0.017083,0.014167,0.005833,0.001250,0.061250,0.103333,0.327083,0.342500,0.065833,0.032083,0.000000
ADV,0.033467,0.135208,0.085676,0.322624,0.016064,0.017403,0.119143,0.073628,0.045515,0.025435,0.117805,0.008032
VERB,0.233073,0.088612,0.077379,0.165679,0.034321,0.039626,0.069579,0.131669,0.109828,0.020281,0.026209,0.003744
PRT,0.012690,0.017766,0.010152,0.384518,0.001269,0.024112,0.077411,0.105330,0.277919,0.044416,0.039340,0.005076
PRON,0.106259,0.024745,0.039301,0.455604,0.018923,0.014556,0.077147,0.004367,0.208151,0.008734,0.037846,0.004367
ADJ,0.015152,0.071338,0.005682,0.006944,0.009470,0.001263,0.070707,0.005682,0.707071,0.021465,0.065657,0.019571
DET,0.040284,0.009005,0.013744,0.030332,0.000474,0.002370,0.224645,0.006161,0.638389,0.017536,0.016588,0.000474
NOUN,0.029576,0.179756,0.017085,0.152046,0.046662,0.006748,0.011055,0.010912,0.261019,0.006030,0.237473,0.041637
NUM,0.176912,0.053973,0.002999,0.019490,0.031484,0.000000,0.040480,0.004498,0.349325,0.164918,0.134933,0.020990


## Viterbi algorithm implementation for POS tagging

In [16]:
def Viterbi_algorithm(words, train_data = train_pairs):
    state = []
    T = list(set([pair[1] for pair in train_data]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = transition_df.loc['.', tag]
            else:
                transition_p = transition_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = calc_emmision_probability(words[key], tag)[0]/calc_emmision_probability(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

## Testing and evaluating the algorithm

In [17]:
random.seed(1234)

# list of tagged words
test_run_base = [tup for sent in test_set for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_set for tup in sent]

In [18]:
tagged_seq = Viterbi_algorithm(test_tagged_words)
print(tagged_seq)

[('Commonwealth', 'NOUN'), ('Edison', 'NOUN'), ('now', 'ADV'), ('faces', 'X'), ('an', 'DET'), ('additional', 'ADJ'), ('court-ordered', 'X'), ('refund', 'VERB'), ('on', 'ADP'), ('its', 'PRON'), ('summer\\/winter', 'X'), ('rate', 'NOUN'), ('differential', 'X'), ('collections', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('Illinois', 'NOUN'), ('Appellate', 'X'), ('Court', 'NOUN'), ('has', 'VERB'), ('estimated', 'VERB'), ('*T*-1', 'X'), ('at', 'ADP'), ('$', '.'), ('140', 'X'), ('million', 'NUM'), ('*U*', 'X'), ('.', '.'), ('Northeast', 'NOUN'), ('said', 'VERB'), ('0', 'X'), ('it', 'PRON'), ('would', 'VERB'), ('refile', 'X'), ('its', 'PRON'), ('request', 'NOUN'), ('and', 'CONJ'), ('still', 'ADV'), ('hopes', 'X'), ('for', 'ADP'), ('an', 'DET'), ('expedited', 'X'), ('review', 'VERB'), ('by', 'ADP'), ('the', 'DET'), ('FERC', 'X'), ('so', 'ADV'), ('that', 'ADP'), ('it', 'PRON'), ('could', 'VERB'), ('complete', 'VERB'), ('the', 'DET'), ('purchase', 'NOUN'), ('by', 'ADP'), ('next', 'ADJ'), ('summer

In [19]:
# Calculating the accuracy of the vanilla viterbi algorithm

check = [i for i, j in zip(tagged_seq, test_run_base) if i == j]

acc = len(check)/len(tagged_seq)

print("The accuracy of Viterbi Algorithm is -", acc)

The accuracy of Viterbi Algorithm is - 0.8196078431372549


In [20]:
# Testing on random sentence

pos_result = Viterbi_algorithm(['The', 'company', 'opened', 'in', 'July'])
print(pos_result)

[('The', 'DET'), ('company', 'NOUN'), ('opened', 'VERB'), ('in', 'ADP'), ('July', 'NOUN')]


## Modifing algorithm to handle unknown words
If a word is not present in the training vocabulary (unknown word), its emission probability will be ZERO. In that case the state probability will inturn be zero. Thus in cases where the algorithm encounters a word which it has not see previously, we can omit the emission probability and only consider the transition probability.

The modified algorithm -

- If(word in unknown) then
    - consider only the transition_p (emission probability being 0)
- else
    - consider both the emission and transition probability.

In [21]:
def Viterbi_for_Unknown_Words(words, train_bag = train_pairs):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = transition_df.loc['.', tag]
            else:
                transition_p = transition_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = calc_emmision_probability(words[key], tag)[0]/calc_emmision_probability(words[key], tag)[1]
            
            # modification to the original vanilla viterbi algorithm 
            # to consider unknown words
            if word not in training_vocabulary_set:
                state_probability = transition_p
            else:
                state_probability = emission_p * transition_p
                
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [22]:
# tagging the test sentences
unknown_word_tagged_seq = Viterbi_for_Unknown_Words(test_tagged_words)

In [23]:
# Calculating accuracy
unknown_word_check = [i for i, j in zip(unknown_word_tagged_seq, test_run_base) if i == j]

optimized_viterbi_accuracy = len(unknown_word_check)/len(unknown_word_tagged_seq)

print("The accuracy of the Vanilla Viterbi Algorithm after modification to handle unknown words is -", optimized_viterbi_accuracy)

The accuracy of the Vanilla Viterbi Algorithm after modification to handle unknown words is - 0.8941176470588236
